In [41]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from sub import subMNIST

In [42]:
trainset_labeled = pickle.load(open("train_labeled.p", "rb")) 
train_loader  = torch.utils.data.DataLoader(trainset_labeled, batch_size=64, shuffle=True, num_workers=2)

validset = pickle.load(open("validation.p", "rb"))
valid_loader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=True, num_workers=2)

In [43]:
#utility functions
def rand(x, level = 1e-3):
    return x + torch.randn(x.size()) * level

def norm_weights_2d(size):
    return BatchNorm2d(nn.Linear(size))
def norm_weights_1d(size):
    return BatchNorm2d(nn.Linear(size))

def reluN(x, level = 1e-3):
    y = x + torch.randn(x.size()) * level

In [56]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = [1000, 600, 300, 10]
        self.enc1 = nn.Linear(28*28, self.layers[0])
        self.lat1 = nn.Linear(self.layers[0],self.layers[0])
        self.BatchNorm1 = torch.nn.BatchNorm1d(self.layers[0])
        
        self.enc2 = nn.Linear(self.layers[0],self.layers[1])
        self.lat2 = nn.Linear(self.layers[1],self.layers[1])
        self.dec21 = nn.Linear(self.layers[1], self.layers[0])
        self.BatchNorm2 = torch.nn.BatchNorm1d(self.layers[1])
        self.DecNorm2 = torch.nn.BatchNorm1d(self.layers[0])
        
        self.enc3 = nn.Linear(self.layers[1], self.layers[2])
        self.lat3 = nn.Linear(self.layers[2],self.layers[2])
        self.dec32 = nn.Linear(self.layers[2], self.layers[1])
        self.BatchNorm3 = torch.nn.BatchNorm1d(self.layers[2])
        self.DecNorm3 = torch.nn.BatchNorm1d(self.layers[1])
        
        self.enc4 = nn.Linear(self.layers[2], self.layers[3])
        self.dec43 = nn.Linear(self.layers[3], self.layers[2])
        self.BatchNorm4 = torch.nn.BatchNorm1d(self.layers[3])
        self.DecNorm4 = torch.nn.BatchNorm1d(self.layers[2])
        
        self.batch_size = 64
        self.weights = [self.enc1,self.enc2,self.enc3,self.enc4,
                        self.lat1,self.lat2,self.lat3,
                        self.dec21, self.dec32, self.dec43]
        

    def forward(self, x, v = 0, noise=1e-3):
        self.eps = noise
        self.batch_size = x.size()[0]
        #enc= F.relu(self.enc4(F.relu(self.enc3(F.relu(self.enc2(F.relu(self.enc1(x))))))))
        
        enc1 = self.BatchNorm1(self.enc1(x))
        enc1out = F.relu(enc1 + self.BatchNorm1.weight.unsqueeze(0).expand(self.batch_size, self.layers[0]) *
                     torch.autograd.Variable(torch.randn(enc1.size()) * self.eps) + \
                     self.BatchNorm1.bias.unsqueeze(0).expand(self.batch_size, self.layers[0]))
        enc2 = self.BatchNorm2(self.enc2(enc1out))
        enc2out = F.relu(enc2 + self.BatchNorm2.weight.unsqueeze(0).expand(self.batch_size, self.layers[1]) *
                     torch.autograd.Variable(torch.randn(enc2.size()) * self.eps) + \
                     self.BatchNorm2.bias.unsqueeze(0).expand(self.batch_size, self.layers[1]))
        enc3 = self.BatchNorm3(self.enc3(enc2out))
        enc3out = F.relu(enc3 + self.BatchNorm3.weight.unsqueeze(0).expand(self.batch_size, self.layers[2]) *
                     torch.autograd.Variable(torch.randn(enc3.size()) * self.eps) + \
                     self.BatchNorm3.bias.unsqueeze(0).expand(self.batch_size, self.layers[2]))
        enc4 = self.BatchNorm4(self.enc4(enc3out))
        enc4out = F.relu(enc4 + self.BatchNorm4.weight.unsqueeze(0).expand(self.batch_size, self.layers[3]) *
                     torch.autograd.Variable(torch.randn(enc4.size()) * self.eps) + \
                     self.BatchNorm4.bias.unsqueeze(0).expand(self.batch_size, self.layers[3]))
        
        
        GT1 = F.relu(self.BatchNorm1(self.enc1(x)))
        GT2 = F.relu(self.BatchNorm2(self.enc2(GT1)))
        GT3 = F.relu(self.BatchNorm3(self.enc3(GT2)))
        GT4 = F.relu(self.BatchNorm4(self.enc4(GT3)))
        
        decin = F.softmax(enc4out)
        
        dec4 = self.dec43(decin)+self.lat3(enc3out)
        dec4out = F.relu(dec4 + self.DecNorm4.weight.unsqueeze(0).expand(self.batch_size, self.layers[2]) *
                     torch.autograd.Variable(torch.randn(dec4.size()) * self.eps) + \
                     self.DecNorm4.bias.unsqueeze(0).expand(self.batch_size, self.layers[2]))
        
        dec3 = self.dec32(dec4out)+self.lat2(enc2out)
        dec3out = F.relu(dec3 + self.DecNorm3.weight.unsqueeze(0).expand(self.batch_size, self.layers[1]) *
                     torch.autograd.Variable(torch.randn(dec3.size()) * self.eps) + \
                     self.DecNorm3.bias.unsqueeze(0).expand(self.batch_size, self.layers[1]))
        
        dec2 = self.dec21(dec3out)+self.lat1(enc1out)
        dec2out = F.relu(dec2 + self.DecNorm2.weight.unsqueeze(0).expand(self.batch_size, self.layers[0]) *
                     torch.autograd.Variable(torch.randn(dec2.size()) * self.eps) + \
                     self.DecNorm2.bias.unsqueeze(0).expand(self.batch_size, self.layers[0]))
        
        reg=0
        for w in self.weights:
            reg += (w.weight**2).mean()/100
        
        return F.log_softmax(enc4out), 1*((GT3 - dec4)**2).mean() +1*((GT2 - dec3)**2).mean() + \
                1*((GT1 - dec2)**2).mean() + reg

model = Net()
params = list(model.parameters())

if 0:
    print(model)
    print('Models has {} learnable paramater:'.format(len(params)))
    [print('parameter {} has a size of {}'.format(i+1, params[i].size())) for i in range(len(params))]

In [57]:
(model.weights[0].weight**2).mean()

Variable containing:
1.00000e-04 *
  4.2563
[torch.FloatTensor of size 1]

In [58]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [59]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target) # Wrap them in Variable 
        optimizer.zero_grad() # Zero the parameter gradients
        data = data.view(data.size()[0], 28*28)
        outputs = model(data,noise=1e-2) # Forward 
        output = outputs[0]
        US = outputs[1]
        loss = F.nll_loss(output, target) + US*.1
        loss.backward() 
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [60]:
def test(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:

        data, target = Variable(data, volatile=True), Variable(target)
        data = data.view(data.size()[0], 28*28)
        outputs = model(data, noise = 0)
        output = outputs[0]
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [61]:
for epoch in range(1, 30):
    train(epoch)
    test(epoch, valid_loader)

Train Epoch: 1 [0/3000 (0%)]	Loss: 2.397224
Train Epoch: 1 [640/3000 (21%)]	Loss: 1.519292
Train Epoch: 1 [1280/3000 (43%)]	Loss: 1.349676
Train Epoch: 1 [1920/3000 (64%)]	Loss: 1.206272
Train Epoch: 1 [2560/3000 (85%)]	Loss: 1.036737

Test set: Average loss: 0.7336, Accuracy: 8942/10000 (89%)

Train Epoch: 2 [0/3000 (0%)]	Loss: 0.859665
Train Epoch: 2 [640/3000 (21%)]	Loss: 0.651262
Train Epoch: 2 [1280/3000 (43%)]	Loss: 0.693339
Train Epoch: 2 [1920/3000 (64%)]	Loss: 0.505255
Train Epoch: 2 [2560/3000 (85%)]	Loss: 0.446675

Test set: Average loss: 0.4306, Accuracy: 9278/10000 (93%)

Train Epoch: 3 [0/3000 (0%)]	Loss: 0.394293
Train Epoch: 3 [640/3000 (21%)]	Loss: 0.412965
Train Epoch: 3 [1280/3000 (43%)]	Loss: 0.304297
Train Epoch: 3 [1920/3000 (64%)]	Loss: 0.339598
Train Epoch: 3 [2560/3000 (85%)]	Loss: 0.257155

Test set: Average loss: 0.3471, Accuracy: 9367/10000 (94%)

Train Epoch: 4 [0/3000 (0%)]	Loss: 0.294940
Train Epoch: 4 [640/3000 (21%)]	Loss: 0.221244
Train Epoch: 4 [1280/